In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM,Conv1D,Conv2D,MaxPooling2D,MaxPooling1D,Flatten,Bidirectional
from keras.layers import Dense, Dropout

import mlflow

In [2]:
import pathlib
mlflow_path = pathlib.Path('../../mlruns').resolve()

In [3]:
mlflow.set_tracking_uri(mlflow_path)
mlflow.tensorflow.autolog()
tf.keras.utils.set_random_seed(42)  # sets seeds for base-python, numpy and tf
tf.config.experimental.enable_op_determinism()

In [4]:
df_2022 = pd.read_csv('../../Tweet_Sentiment/THYAO/2022_final.csv').drop(['fark', 'tweet_count'], axis= 1).set_index('Unnamed: 0').sort_index()
df_2021 = pd.read_csv('../../Tweet_Sentiment/THYAO/2021_final.csv').drop(['fark', 'tweet_count'], axis= 1).set_index('Unnamed: 0').sort_index()

In [5]:
df = pd.concat([df_2022, df_2021]).sort_index()
df

,şimdi,açılış,avg_polar
Unnamed: 0,,,
2021-01-04,12.68,12.97,0.715789
2021-01-05,12.62,12.64,0.729469
2021-01-06,12.69,12.73,0.752577
2021-01-07,12.68,12.74,0.684729
2021-01-08,12.82,12.75,0.711538
...,...,...,...
2022-10-26,98.80,101.20,0.672098
2022-10-27,99.40,99.10,0.638142
2022-10-28,96.95,98.70,0.634686


In [6]:
scaler = MinMaxScaler()
scaler = scaler.fit(df.iloc[:-30, :])
df_for_training_scaled = scaler.transform(df)

scaler_for_inference = MinMaxScaler()
scaler_for_inference.fit(pd.DataFrame(df.iloc[:-30,0]))

MinMaxScaler()

In [7]:
df_for_training_scaled

array([[0.02770428, 0.03244157, 0.53188611],
       [0.02677043, 0.02736777, 0.55893537],
       [0.02785992, 0.02875154, 0.60463083],
       ...,
       [1.33929961, 1.35055351, 0.37151184],
       [1.40700389, 1.33517835, 0.36310474],
       [1.42412451, 1.40590406, 0.37086895]])

In [8]:
#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 5  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future,[0]])

trainX, trainY = np.array(trainX), np.array(trainY)

print('TrainX shape = {}.'.format(trainX.shape))
print('TrainY shape = {}.'.format(trainY.shape))

TrainX shape = (453, 5, 3).
TrainY shape = (453, 1, 1).


In [9]:
X_train_lstm_without_twitter, X_test_lstm_without_twitter, y_train_lstm_without_twitter, y_test_lstm_without_twitter = train_test_split(trainX[:,:,:-1], trainY, test_size=30, shuffle=False)

X_train_lstm_twitter, X_test_lstm_twitter, y_train_lstm_twitter, y_test_lstm_twitter = train_test_split(trainX, trainY, test_size=30, shuffle=False)

X_train_lstm_without_twitter.shape,X_train_lstm_twitter.shape

((423, 5, 2), (423, 5, 3))

In [10]:
def build_model(input_shape):
    tf.keras.utils.set_random_seed(42)  # sets seeds for base-python, numpy and tf
    cnn_lstm_model = Sequential()

    cnn_lstm_model.add(Conv1D(filters=128, kernel_size=2, strides=1, padding='valid', input_shape=input_shape))
    cnn_lstm_model.add(MaxPooling1D(pool_size=2, strides=2))

    cnn_lstm_model.add(Conv1D(filters=64, kernel_size=2, strides=1, padding='valid'))
    cnn_lstm_model.add(MaxPooling1D(pool_size=1, strides=2))
    # cnn_lstm_model.add(MaxPooling1D(pool_size=1, strides=2))

    cnn_lstm_model.add(Bidirectional(LSTM(256, return_sequences=True)))
    cnn_lstm_model.add(Dropout(0.2))
    cnn_lstm_model.add(Bidirectional(LSTM(256, return_sequences=True)))
    cnn_lstm_model.add(Dropout(0.2))

    cnn_lstm_model.add(Dense(32, activation='relu'))


    cnn_lstm_model.add(Dense(trainY.shape[2], activation='relu'))

    # cnn_lstm_model.build(input_shape=(trainX.shape[0], trainX.shape[1], trainX.shape[2]))

    cnn_lstm_model.compile(optimizer='adam', loss='mse')
    cnn_lstm_model.summary()
    return cnn_lstm_model

In [ ]:
cnn_lstm_model_without_twitter=build_model((X_train_lstm_without_twitter.shape[1],X_train_lstm_without_twitter.shape[2]))
cnn_lstm_model_twitter=build_model((X_train_lstm_twitter.shape[1],X_train_lstm_twitter.shape[2]))

history_without_twitter = cnn_lstm_model_without_twitter.fit(X_train_lstm_without_twitter, y_train_lstm_without_twitter, epochs=60, batch_size=64, verbose=1)

history_twitter = cnn_lstm_model_twitter.fit(X_train_lstm_twitter, y_train_lstm_twitter, epochs=60, batch_size=64, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 128)            640       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2, 128)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1, 64)             16448     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 1, 64)            0         
 1D)                                                             
                                                                 
 bidirectional (Bidirectiona  (None, 1, 512)           657408    
 l)                                                              
                                                        

2022/12/16 17:33:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'fba4d90b46bd4bb4b40bb4657b05c121', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/60
7/7 [==============================] - 10s 57ms/step - loss: 0.0795
Epoch 2/60
7/7 [==============================] - 0s 25ms/step - loss: 0.0145
Epoch 3/60
7/7 [==============================] - 0s 19ms/step - loss: 0.0074
Epoch 4/60
7/7 [==============================] - 0s 19ms/step - loss: 0.0021
Epoch 5/60
7/7 [==============================] - 0s 12ms/step - loss: 0.0015
Epoch 6/60
7/7 [==============================] - 0s 12ms/step - loss: 0.0012
Epoch 7/60
7/7 [==============================] - 0s 11ms/step - loss: 7.8163e-04
Epoch 8/60
7/7 [==============================] - 0s 12ms/step - loss: 7.4141e-04
Epoch 9/60
7/7 [==============================] - 0s 12ms/step - loss: 7.6139e-04
Epoch 10/60
7/7 [==============================] - 0s 11ms/step - loss: 6.3845e-04
Epoch 11/60
7/7 [==============================] - 0s 11ms/step - loss: 6.7453e-04
Epoch 12/60
7/7 [==============================] - 0s 11ms/step - loss: 7.6096e-04
Epoch 13/60
7/7 [===================

2022/12/16 17:33:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\assas\anaconda3\envs\tf\lib\site-packages\mlflow\tensorflow\__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


INFO:tensorflow:Assets written to: C:\Users\assas\AppData\Local\Temp\tmpqdxg5yq3\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\assas\AppData\Local\Temp\tmpqdxg5yq3\model\data\model\assets
2022/12/16 17:34:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\assas\anaconda3\envs\tf\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
2022/12/16 17:34:28 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '6ca12324132b4ed3ad06853da9b25775', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/60
7/7 [==============================] - 4s 66ms/step - loss: 0.0808
Epoch 2/60
7/7 [==============================] - 0s 21ms/step - loss: 0.0140
Epoch 3/60
7/7 [==============================] - 0s 20ms/step - loss: 0.0055
Epoch 4/60
7/7 [==============================] - 0s 20ms/step - loss: 0.0030
Epoch 5/60
7/7 [==============================] - 0s 22ms/step - loss: 0.0019
Epoch 6/60
7/7 [==============================] - 0s 22ms/step - loss: 0.0015
Epoch 7/60
7/7 [==============================] - 0s 23ms/step - loss: 0.0012
Epoch 8/60
7/7 [==============================] - 0s 23ms/step - loss: 0.0010
Epoch 9/60
7/7 [==============================] - 0s 24ms/step - loss: 0.0010
Epoch 10/60
7/7 [==============================] - 0s 23ms/step - loss: 8.6238e-04
Epoch 11/60
7/7 [==============================] - 0s 23ms/step - loss: 8.3536e-04
Epoch 12/60
7/7 [==============================] - 0s 21ms/step - loss: 8.7795e-04
Epoch 13/60
7/7 [==============================] 

2022/12/16 17:34:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\assas\anaconda3\envs\tf\lib\site-packages\mlflow\tensorflow\__init__.py:189: UserWarning: The pyfunc inference behavior of Keras models logged with signatures differs from the behavior of Keras models logged without signatures. Specifically, when a signature is present, passing a Pandas DataFrame as input to the pyfunc `predict()` API produces an `ndarray` (for single-output models) or a dictionary of `str -> ndarray`: (for multi-output models). In contrast, when a signature is *not* present, `predict()` produces a Pandas DataFrame output in response to a Pandas DataFrame input."


In [ ]:
plt.figure(figsize=(20,5))
plt.plot(history_twitter.history['loss'], label='Training loss')
plt.title('Training loss')
plt.legend()

In [ ]:
training_dates= df.index[:X_train_lstm_without_twitter.shape[0]]
#Make prediction
training_prediction_without_twitter = cnn_lstm_model_without_twitter.predict(X_train_lstm_without_twitter)

training_prediction_twitter = cnn_lstm_model_twitter.predict(X_train_lstm_twitter)

training_prediction_without_twitter=training_prediction_without_twitter.reshape(training_prediction_without_twitter.shape[0], training_prediction_without_twitter.shape[2])

training_prediction_twitter=training_prediction_twitter.reshape(training_prediction_twitter.shape[0], training_prediction_twitter.shape[2])

y_train_pred_lstm_without_twitter = scaler_for_inference.inverse_transform(training_prediction_without_twitter)

y_train_pred_lstm_twitter = scaler_for_inference.inverse_transform(training_prediction_twitter)

y_train_lstm_reshaped_without_twitter=y_train_lstm_without_twitter.reshape(y_train_lstm_without_twitter.shape[0], y_train_lstm_without_twitter.shape[2])

y_train_actual_lstm = scaler_for_inference.inverse_transform(y_train_lstm_reshaped_without_twitter)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt

def plot_predictions_with_dates (type,twitter,dates,y_actual_lstm,y_pred_lstm):
    predicted_features=['şimdi']
    for i,predicted_feature in enumerate(predicted_features):
        plt.figure(figsize=(15,6))
        if twitter :
            plt.title(f'LSTM {type} prediction of {predicted_feature} feature After adding twitter sentiment analysis')
        else:
            plt.title(f'LSTM {type} prediction of {predicted_feature} feature without twitter sentiment analysis')
        sns.lineplot(x=dates, y=y_actual_lstm[:,i],label='Actual')
        sns.lineplot(x=dates, y=y_pred_lstm[:, i], label='Predicted')
        plt.show()
        error=mean_squared_error(y_actual_lstm[:,i], y_pred_lstm[:, i])
        print(f'Mean square error for {predicted_feature} ={error}')
    print('Total mean square error', mean_squared_error(y_actual_lstm, y_pred_lstm))

In [ ]:
testing_dates= df.index[-X_test_lstm_without_twitter.shape[0]:]
#Make prediction
testing_prediction_without_twitter = cnn_lstm_model_without_twitter.predict(X_test_lstm_without_twitter)
testing_prediction_twitter = cnn_lstm_model_twitter.predict(X_test_lstm_twitter)

testing_prediction_without_twitter=testing_prediction_without_twitter.reshape(testing_prediction_without_twitter.shape[0], testing_prediction_without_twitter.shape[2])
testing_prediction_twitter=testing_prediction_twitter.reshape(testing_prediction_twitter.shape[0], testing_prediction_twitter.shape[2])

y_test_pred_lstm_without_twitter = scaler_for_inference.inverse_transform(testing_prediction_without_twitter)
y_test_pred_lstm_twitter = scaler_for_inference.inverse_transform(testing_prediction_twitter)

y_test_actual_lstm_reshaped_without_twitter=y_test_lstm_without_twitter.reshape(y_test_lstm_without_twitter.shape[0], y_test_lstm_without_twitter.shape[2])


y_test_actual_lstm = scaler_for_inference.inverse_transform(y_test_actual_lstm_reshaped_without_twitter)

In [ ]:
training_dates= df.index[:X_train_lstm_twitter.shape[0]]
plot_predictions_with_dates('Training',False,training_dates,y_train_actual_lstm ,y_train_pred_lstm_without_twitter)

In [ ]:
training_dates= df.index[:X_train_lstm_twitter.shape[0]]
plot_predictions_with_dates('Training',True,training_dates,y_train_actual_lstm ,y_train_pred_lstm_twitter)

In [ ]:
plot_predictions_with_dates('Testing',False,testing_dates,y_test_actual_lstm,y_test_pred_lstm_without_twitter)

In [ ]:
plot_predictions_with_dates('Testing',True,testing_dates,y_test_actual_lstm,y_test_pred_lstm_twitter)

In [ ]:
cnn_lstm_model_twitter.save('cnn_lstm_twitter.h5')

In [ ]:
from pickle import dump
dump(scaler, open('in_scaler.pkl', 'wb'))
dump(scaler_for_inference, open('out_scaler.pkl', 'wb'))